In [9]:
import polars as pl

# df = pl.read_csv("../data/hotel_bookings.csv", ignore_errors=True)


In [10]:
# df = df.filter(pl.col("arrival_date_year") > 2015).groupby("hotel").sum()
# df


In [11]:
# q = (pl.scan_csv("../data/hotel_bookings.csv", ignore_errors=True)
#     .filter(pl.col("arrival_date_year") > 2015)
#     .groupby("hotel")
#     .agg(pl.col("*").sum())
# )
# df = q.collect()
# df


In [12]:
import numpy as np

np.random.seed(12)

df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5, 6],
        "names": ["foo", "ham", "spam", "foo", "foo", "spam"],
        "random": np.random.rand(6),
        "random2": np.random.rand(6),
        "groups": ["A", "A", "B", "C", "A", "A"],
    }
)
print(df)


shape: (6, 5)
┌──────┬───────┬──────────┬──────────┬────────┐
│ nrs  ┆ names ┆ random   ┆ random2  ┆ groups │
│ ---  ┆ ---   ┆ ---      ┆ ---      ┆ ---    │
│ i64  ┆ str   ┆ f64      ┆ f64      ┆ str    │
╞══════╪═══════╪══════════╪══════════╪════════╡
│ 1    ┆ foo   ┆ 0.154163 ┆ 0.900715 ┆ A      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 2    ┆ ham   ┆ 0.74     ┆ 0.033421 ┆ A      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 3    ┆ spam  ┆ 0.263315 ┆ 0.956949 ┆ B      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ null ┆ foo   ┆ 0.533739 ┆ 0.137209 ┆ C      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 5    ┆ foo   ┆ 0.014575 ┆ 0.283828 ┆ A      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 6    ┆ spam  ┆ 0.918747 ┆ 0.606083 ┆ A      │
└──────┴───────┴──────────┴──────────┴────────┘


In [13]:
df2 = df[
    [
        # pl.sum("random").alias("sum_function"),
        # pl.sum(pl.Float64),
        pl.all()
        .exclude(["nrs", "names", "groups"])
        .sum()
        # pl.sum("^random.*$")
    ]
]
print(df2)


shape: (1, 2)
┌──────────┬──────────┐
│ random   ┆ random2  │
│ ---      ┆ ---      │
│ f64      ┆ f64      │
╞══════════╪══════════╡
│ 2.624589 ┆ 2.918206 │
└──────────┴──────────┘


In [14]:
df3 = df[
    [
        pl.col("random").count().alias("count_method"),
        pl.count("random").alias("count_function"),
    ]
]
print(df3)


shape: (1, 2)
┌──────────────┬────────────────┐
│ count_method ┆ count_function │
│ ---          ┆ ---            │
│ u32          ┆ u32            │
╞══════════════╪════════════════╡
│ 6            ┆ 6              │
└──────────────┴────────────────┘


In [17]:
df4 = df.select(
    [
        pl.when(pl.col("random") > 0.5)
        .then(0)
        .otherwise(pl.col("random") * pl.sum("nrs"))
        .alias("binary_function")
    ]
)
print(df4)


shape: (6, 1)
┌─────────────────┐
│ binary_function │
│ ---             │
│ f64             │
╞═════════════════╡
│ 2.620768        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 0.0             │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4.476355        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 0.0             │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 0.247774        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 0.0             │
└─────────────────┘


In [46]:
df_conditional = df.select(
    [
        pl.all(),
        pl.when(pl.col("names") == "foo")
        .then(999)
        # .when(pl.col("names") == "ham")
        # .then(pl.col("nrs") * pl.col("random"))
        .when((pl.col("names") == "spam") & (pl.col("nrs") < 4))
        .then(pl.col("nrs") * 2)
        .otherwise("other")
        .alias("names_transformed"),
    ]
)

print(df_conditional)


shape: (6, 6)
┌──────┬───────┬──────────┬──────────┬────────┬───────────────────┐
│ nrs  ┆ names ┆ random   ┆ random2  ┆ groups ┆ names_transformed │
│ ---  ┆ ---   ┆ ---      ┆ ---      ┆ ---    ┆ ---               │
│ i64  ┆ str   ┆ f64      ┆ f64      ┆ str    ┆ str               │
╞══════╪═══════╪══════════╪══════════╪════════╪═══════════════════╡
│ 1    ┆ foo   ┆ 0.154163 ┆ 0.900715 ┆ A      ┆ 999               │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ ham   ┆ 0.74     ┆ 0.033421 ┆ A      ┆ null              │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ spam  ┆ 0.263315 ┆ 0.956949 ┆ B      ┆ 6                 │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ null ┆ foo   ┆ 0.533739 ┆ 0.137209 ┆ C      ┆ 999               │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5    ┆ foo   ┆ 0.014575 ┆ 0.283828 ┆ A      ┆ 999               │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌

In [8]:
df5 = df[
    [
        # pl.all().exclude("^random.*$"),
        pl.col("*").exclude("^random.*$"),
        pl.col("names").list().over("groups").alias("names/groups"),
        pl.col("names").unique().over("groups").alias("unique_names/groups"),
    ]
]
print(df5)


shape: (6, 5)
┌──────┬───────┬────────┬────────────────────────────┬────────────────────────┐
│ nrs  ┆ names ┆ groups ┆ names/groups               ┆ unique_names/groups    │
│ ---  ┆ ---   ┆ ---    ┆ ---                        ┆ ---                    │
│ i64  ┆ str   ┆ str    ┆ list [str]                 ┆ list [str]             │
╞══════╪═══════╪════════╪════════════════════════════╪════════════════════════╡
│ 1    ┆ foo   ┆ A      ┆ ["foo", "ham", ... "spam"] ┆ ["ham", "spam", "foo"] │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2    ┆ ham   ┆ A      ┆ ["foo", "ham", ... "spam"] ┆ ["ham", "spam", "foo"] │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3    ┆ spam  ┆ B      ┆ ["spam"]                   ┆ ["spam"]               │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ null ┆ foo   ┆ C      ┆ ["foo"]                    ┆ ["foo"]                │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌

In [59]:
import numpy as np

np.random.seed(12)

df_small = pl.DataFrame(
    {
        "nrs": [1, 2, 3],
        "names": ["foo", "spam", "spam"],
    }
)
print(df)


shape: (6, 5)
┌──────┬───────┬──────────┬──────────┬────────┐
│ nrs  ┆ names ┆ random   ┆ random2  ┆ groups │
│ ---  ┆ ---   ┆ ---      ┆ ---      ┆ ---    │
│ i64  ┆ str   ┆ f64      ┆ f64      ┆ str    │
╞══════╪═══════╪══════════╪══════════╪════════╡
│ 1    ┆ foo   ┆ 0.154163 ┆ 0.900715 ┆ A      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 2    ┆ ham   ┆ 0.74     ┆ 0.033421 ┆ A      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 3    ┆ spam  ┆ 0.263315 ┆ 0.956949 ┆ B      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ null ┆ foo   ┆ 0.533739 ┆ 0.137209 ┆ C      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 5    ┆ foo   ┆ 0.014575 ┆ 0.283828 ┆ A      │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ 6    ┆ spam  ┆ 0.918747 ┆ 0.606083 ┆ A      │
└──────┴───────┴──────────┴──────────┴────────┘


In [67]:
df_conditional = df_small.with_columns(
    [
        pl.when((pl.col("names") == "spam") & (pl.col("nrs") == 2))
        .then(pl.col("nrs") * 2)
        .otherwise("other")
        .alias("new_col"),
    ]
)

print(df_conditional)


shape: (3, 3)
┌─────┬───────┬─────────┐
│ nrs ┆ names ┆ new_col │
│ --- ┆ ---   ┆ ---     │
│ i64 ┆ str   ┆ i64     │
╞═════╪═══════╪═════════╡
│ 1   ┆ foo   ┆ null    │
├╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ 2   ┆ spam  ┆ 4       │
├╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ 3   ┆ spam  ┆ null    │
└─────┴───────┴─────────┘


In [69]:
pl.version()

'0.12.22'